## Ordinary Differential Equations
John Lochner & Andrew Sos 

Homework 3

# Number 2
The central finite difference method was derived. This method is efficient in optimizing a numerical solution to a differential equation using the central node. The boundary conditions for our original central finite differnece method were as followed:
$\psi(a) = \psi(b)$ and $\psi'(a) = \psi'(b)$.



# Key Equations
The first condition is discretized as 
$$
  \psi(a) = \psi(b) \Longrightarrow \psi(x_0) = \psi(x_N).
$$

Whereas, the second condition is discretized as 
$$
    \psi'(a) = \psi'(b) \Longrightarrow
$$

$$
    \frac{\psi(x_1) - \psi(x_0)}{\Delta x} = \frac{\psi(x_{N+1}) - \psi(x_{N})}{\Delta x}
    \Longrightarrow
$$
$$
    \psi(x_1) - \psi(x_N) = \psi(x_{N+1}) - \psi(x_{N})
    \Longrightarrow
$$
$$
    \psi(x_1) = \psi(x_{N+1}).
$$

## Analytical Solution
Time independent Schrodinger EQ is discretized as:
$$
\begin{align}
	-\frac{\psi(x_{k+1}) - 2\psi(x_{k})  + \psi(x_{k - 1})}{2\Delta x^2} + U(x_k)  \psi(x_{k}) = E  \psi(x_{k}),
	\qquad k=1,1,2,\ldots,N,
\end{align}
$$
Matrix representation of time independent Schrodinger eqaution has periodic boundary condition:

(1)   $$\psi(x_{0}) = \psi(x_{N+1}) = 0$$

In discrete form,

(2)  $$\psi'(a) = \psi'(b)$$
and
$$
    \psi(x_1) = \psi(x_{N+1}).
$$
Central finite difference method:
$$
\begin{align}
	f'(x) &\approx [ f(x+\Delta x/2) - f(x-\Delta x/2) ] / \Delta x = f'(x) + O\left( \Delta x^2 \right) \Longrightarrow {CentralFinitDiffApprox} \\
	f''(x) &\approx  [ f'(x+\Delta x/2) - f'(x-\Delta x/2) ] / \Delta x  = [ f(x + \Delta x) - 2f(x) + f(x- \Delta x)] / \Delta x^2  + O\left( \Delta x^2 \right). 
\end{align}
$$


Listing K values, then adjusting for boundary conditions yields a pattern:

K=1$$-\frac{\psi(x_2) - 2\psi(x_1)+\psi(x_0)}{2\Delta x^2} + U(x_1)  \psi(x_1) = E \psi(x_1)$$

K=2$$\frac{\psi(x_3) - 2\psi(x_2)+\psi(x_1)}{2\Delta x^2} + U(x_2)  \psi(x_2) = E  \psi(x_2)$$
K=...$$...$$\
K=$N-1$ $$\frac{\psi(x_N) - 2\psi(x_{N-1})+\psi(x_{N-2})}{2\Delta x^2} + U(x_{N-1})  \psi(x_{N-1}) = E  \psi(x_{N-1})$$
K=$N$ $$\frac{\psi(x_{N+1}) - 2\psi(x_{N})+\psi(x_{N-1})}{2\Delta x^2} + U(x_N)  \psi(x_N) = E  \psi(x_N)=\frac{\psi(x_{1}) - 2\psi(x_{N})+\psi(x_{N-1})}{2\Delta x^2} + U(x_N)  \psi(x_N)$$

This pattern is represented as a matrix and is the Hermitian finite dimensional approximation of the above Hamiltonian:
\
\
$(\frac{-1}{2\Delta x^2})$ *
$\begin{pmatrix}
-2 & 1       & 0      & 0     & \cdots & 0 & 0  & 0  & 1 \\
1  & -2      & 1      & 0     & \cdots & 0 & 0  & 0  & 0 \\ 
 \ddots  &  \ddots & \ddots & \ddots &    \ddots     &  \ddots  &  \ddots   &  \ddots   &  \ddots  \\
0  & 0       & 0      & 0     & \cdots & 0 & 1  & -2 & 1  \\
1  & 0       & 0      & 0     & \cdots & 0 & 0  & 1  & -2 \\
\end{pmatrix}$
+
$\begin{pmatrix}
U(x_1) &       &       &       &       &          &           &         \\   
       &U(x_2)&       &       &       &          &           &          \\   
       &      &       & \ddots&       &          &           &          \\ 
       &      &       &       & \ddots&          &           &          \\        
       &      &       &       &       &          & U(x_{N-1})&          \\   
       &      &       &       &       &          &           & U(x_{N}) \\
       \end{pmatrix}$
       $\begin{pmatrix}
\Psi(x_1) \\
\Psi(x_2) \\
\Psi(x_{N-1}) \\
\Psi(x_{N}) \\ \end{pmatrix}$= E
$\begin{pmatrix}
\Psi(x_1) \\
\Psi(x_2) \\
\Psi(x_{N-1}) \\
\Psi(x_{N}) \\ \end{pmatrix}$

Overall, we were able to show how the boundry conditions for central difference method's boundry's change the conditions of the matrix.  